In [12]:
import re

vertex_count = 0;
class Graph:
    def __init__(self):
        self.graph = {}

    def add_edge(self, node, parent_set, cost):
        if node not in self.graph:
            self.graph[node] = []
        self.graph[node].append((parent_set, cost))
        
    '''def add_edge(self, node, parent_set, cost):
        if node not in self.graph:
            self.graph[node] = []
        
        if isinstance(parent_set, int):
            parent_set = {parent_set}
        
        for parent in parent_set:
            self.graph[node].append((parent, cost))''' 

    def bfs(self, start_node):
        visited = set()
        queue = [start_node]
        ordering = []

        while queue:
            current_node = queue.pop(0)
            if current_node not in visited:
                visited.add(current_node)
                ordering.append(current_node)

                neighbors = self.graph.get(current_node, [])
                neighbors = [(neighbor, cost) for neighbor, cost in neighbors if cost is not None]
                neighbors.sort(key=lambda x: x[1])
                for neighbor, _ in neighbors:
                    if neighbor not in visited:
                        queue.append(neighbor)

        return ordering

    def dfs(self, start_node):
        visited = set()
        ordering = []

        def dfs_recursive(node):
            nonlocal visited, ordering
            visited.add(node)
            ordering.append(node)

            neighbors = self.graph.get(node, [])
            neighbors = [(neighbor, cost) for neighbor, cost in neighbors if cost is not None]
            neighbors.sort(key=lambda x: x[1])
            for neighbor, _ in neighbors:
                if neighbor not in visited:
                    dfs_recursive(neighbor)

        dfs_recursive(start_node)
        return ordering

    def ucs(self, start_node):
        visited = set()
        heap = [(0, start_node)]
        ordering = []

        while heap:
            current_cost, current_node = heap.pop(0)
            if current_node not in visited:
                visited.add(current_node)
                ordering.append(current_node)

                neighbors = self.graph.get(current_node, [])
                neighbors = [(neighbor, cost) for neighbor, cost in neighbors if cost is not None]
                neighbors.sort(key=lambda x: x[1])
                for neighbor, cost in neighbors:
                    if neighbor not in visited:
                        heap.append((current_cost + cost, neighbor))
                        heap.sort(key=lambda x: x[0])

        return ordering
    
def calculate_cost(ordering, graph):
    total_cost = 0
    for i in range(len(ordering) - 1):
        current_node = ordering[i]
        next_node = ordering[i + 1]

        #For the first node in the ordering, we add its cost
        if i == 0:
            first_node_cost = next((cost for _, cost in graph.get(current_node, []) if cost is not None), 0)
            #print(f"Adding cost for {current_node}: {first_node_cost}")
            total_cost += first_node_cost

        #Find all possible edges from current_node to next_node
        edges = [(neighbor, cost) for neighbor, cost in graph.get(current_node, []) if neighbor == next_node]

        if len(ordering) == vertex_count and len(edges) >= 1:  # Include the first edge and consider paths with exactly 5 edges
            #Select the edge with the minimum cost
            min_cost = min(cost for _, cost in edges if cost is not None)
            #print(f"Adding cost for transition {current_node} to {next_node}: {min_cost}")
            total_cost += min_cost

    return total_cost


def print_result(ordering, total_cost, method):
    print(f"{method} Ordering: {ordering}")
    print(f"{method} Total Cost: {total_cost}\n")

def find_least_cost_ordering(graph):
    start_nodes = list(graph.graph.keys())
    results = {}

    for start_node in start_nodes:
        bfs_ordering = graph.bfs(start_node)
        dfs_ordering = graph.dfs(start_node)
        ucs_ordering = graph.ucs(start_node)

        bfs_cost = calculate_cost(bfs_ordering, graph.graph)
        dfs_cost = calculate_cost(dfs_ordering, graph.graph)
        ucs_cost = calculate_cost(ucs_ordering, graph.graph)

        # Only consider paths with exactly 5 nodes and 5 edges
        if len(bfs_ordering) == vertex_count and len(set(bfs_ordering)) == vertex_count:
            results["BFS"] = (bfs_ordering, bfs_cost)

        if len(dfs_ordering) == vertex_count and len(set(dfs_ordering)) == vertex_count:
            results["DFS"] = (dfs_ordering, dfs_cost)

        if len(ucs_ordering) == vertex_count and len(set(ucs_ordering)) == vertex_count:
            results["UCS"] = (ucs_ordering, ucs_cost)

    return results



'''def read_data_from_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(',')
            if len(parts) >= 2:
                node, parent_set_str, cost = parts[0], parts[1], parts[2] if len(parts) == 3 else None
                node = int(node)
                cost = float(cost) if cost is not None else None

                # Extracting parent nodes from the parent set
                parent_nodes = set()
                if parent_set_str and parent_set_str != '{}':
                    parent_set = parent_set_str.strip('{}').split(',')
                    for parent in parent_set:
                        # Change this line to directly add integers from the parent set
                        parent_nodes.add(int(parent.strip()))

                data.append((node, parent_nodes, cost))
            else:
                print(f"Skipping invalid line: {line}")

    return data
'''

def read_data_from_file(file_path):
    global vertex_count  # Add this line to declare the use of the global variable

    data = []
    unique_vertices = set()

    with open(file_path, 'r') as file:
        for line in file:
            parts = re.split(r',(?![^{]*\})', line.strip())
            if len(parts) >= 2:
                node, parent_set_str, cost = parts[0], parts[1], parts[2] if len(parts) == 3 else None
                node = int(node)
                cost = float(cost) if cost is not None else None

                # Extracting parent nodes from the parent set
                parent_nodes = set()
                if parent_set_str and parent_set_str != '{}':
                    parent_set = parent_set_str.strip('{}').split(',')
                    for parent in parent_set:
                        parent_nodes.add(int(parent.strip()))

                unique_vertices.add(node)
                unique_vertices.update(parent_nodes)

                data.append((node, parent_nodes, cost))

                #print(f"Read from file: Node={node}, Parent Set={parent_nodes}, Cost={cost}")
            else:
                print(f"Skipping invalid line: {line}")

    vertex_count = len(unique_vertices)
    #print(f"Total unique vertices added: {vertex_count}")
    return data



# Example usage
file_path = "data0.txt"
#file_path = "testdata.txt"
data = read_data_from_file(file_path)

#print("Vertex_count :", vertex_count)

# Create a graph from the given data
graph = Graph()
for node, parent_set, cost in data:
    for parent in parent_set:
        graph.add_edge(parent, node, cost)

# Find and print the least cost ordering for each method individually
results = find_least_cost_ordering(graph)

for method, (ordering, cost) in results.items():
    print_result(ordering, cost, f"{method}")


BFS Ordering: [2, 3, 4, 5, 1]
BFS Total Cost: 446.44599999999997

DFS Ordering: [2, 3, 5, 4, 1]
DFS Total Cost: 434.619

UCS Ordering: [2, 3, 4, 5, 1]
UCS Total Cost: 446.44599999999997

